## Equity pipeline
Use the built-in transformer to normalize trades before building equity.

In [1]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent if not (Path.cwd() / "src").exists() else Path.cwd()
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_PATH in sys.path:", SRC_PATH in map(Path, sys.path))

PROJECT_ROOT: c:\Users\user\Documents\spreader_pro\code\test_visualisation
SRC_PATH in sys.path: True


### Создание датасета OHCL для тестового периода

In [2]:
from data.candle_loader import load_candles  # т.к. файл src/data/candle_loader.py

# укажи путь к файлу с результатами сделок (тот, где "Дата и время" и "Код инструмента")
RESULT_CSV = 'test_result_SP.csv'   # поменяй под себя

# если сырые минутки лежат у тебя в другом месте — можно переопределить
DATA_DIR = r"G:\My Drive\data_fut"  # или r"G:\My Drive\data_fut"

# если справочник шагов не рядом с модулем — тоже можно указать:
STEPS_PATH = PROJECT_ROOT / "src" / "data" / "moex_tickers_steps.csv"

df = load_candles(
    result_csv_path=str(RESULT_CSV),
    data_dir=str(DATA_DIR),
    steps_path=str(STEPS_PATH),
    resample_rule=None,   # или "5T" / "5min"
    save=True,
)

In [6]:
from data.equity_builder import build_equity

candle_csv = PROJECT_ROOT / "test_result_SP_candle_data.csv"
trades_csv = PROJECT_ROOT / "test_result_SP.csv"

equity_df = build_equity(
    candles_csv=candle_csv,
    trades_csv=trades_csv,
    trades_use_transformer=True,
)
equity_df.head()

c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\equity_builder.py:27: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\csv_transformer.py:73: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\equity_builder.py:27: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



,DATE_TIME,OPEN,HIGH,LOW,CLOSE,VOL,1_step_price,date_time,action_price,current_pos,prev_position,deal_price,prev_action_price,delta_price,Pnl,Equity
0,2025-09-17 08:59:00,11.646,11.646,11.646,11.646,193.0,1000.0,2025-09-17 08:59:00,11.646,0,0,NaN,11.646,0.000,0.0,0.0
1,2025-09-17 09:00:00,11.645,11.654,11.640,11.650,1741.0,1000.0,2025-09-17 09:00:00,11.650,1,0,11.65,11.646,0.004,0.0,0.0
2,2025-09-17 09:01:00,11.647,11.664,11.647,11.664,2979.0,1000.0,2025-09-17 09:01:00,11.647,1,1,NaN,11.650,-0.003,-3.0,-3.0
3,2025-09-17 09:02:00,11.662,11.664,11.659,11.661,1947.0,1000.0,2025-09-17 09:02:00,11.659,1,1,NaN,11.647,0.012,12.0,9.0
4,2025-09-17 09:03:00,11.660,11.663,11.660,11.663,1605.0,1000.0,2025-09-17 09:03:00,11.660,1,1,NaN,11.659,0.001,1.0,10.0


In [4]:
equity_df = equity_df.head(2000)

In [7]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scattergl(        # ⚡ ScatterGL — рендер на GPU
    x=equity_df['DATE_TIME'],
    y=equity_df['Equity'],
    mode='lines',
    line=dict(width=1, color='blue')
))
fig.show()



# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# plt.plot(equity_df['DATE_TIME'], equity_df['Equity'])
# plt.title('Equity Over Time')
# plt.xlabel('Date Time')
# plt.ylabel('Equity')
# plt.grid(True)
# plt.xticks(rotation=45)
# plt.tight_layout()